In [112]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [ ]:
#Run Vivado

In [13]:
modules = os.listdir("Modules")

In [14]:
#to segragate top modules from all the modules
top_modules = [mod for mod in modules if mod [:3]=='top']


In [15]:
top_modules

['top_CLA8.v', 'top_CLA16.v', 'top_CLA32.v', 'top_CLA64.v']

In [26]:
os.system("vivado -mode batch -source tcl_add1.tcl")

0

In [22]:
#extra -tclargs is added to take each top module

In [111]:
for filename in top_modules:
    os.mkdir("Impl/"+filename[:-2])
    os.system("vivado -mode batch -source tcl_run.tcl -tclargs {}".format(filename))

FileExistsError: [Errno 17] File exists: 'Impl/top_CLA8'

In [ ]:
#ANalyse Metrics

In [531]:
designs = os.listdir("Impl")
file = ['power.txt','timing.txt','utilization.txt']
col = ['Signal Power (w)', 'Logic Power (w)','Static Power (w)','DC Power (w)','Total Power (w)','DataPath Delay','F7_Muxes','LUT_as_Memory','LUT_as_logic','Slice_LUTs','LUT1','LUT2','LUT3','LUT4','LUT5','LUT6','LUT7','LUT_as_Memory','LUT_as_Logic2','Slice_LUTs2','LUTs_difference']


In [573]:
#power lists
sigdes=[]
slogdes=[]
stpdes =[]
tonpdes = []
dc_power_des =[]
#timing lists
dpddes = []

#utilization lists
muxdes =[]
lutmdes =[]
lutldes =[]
sldes = []
lut1des=[]
lut2des=[]
lut3des=[]
lut4des=[]
lut5des=[]
lut6des=[]
lut7des=[]
lutmdes2=[]
LUT_as_Logic2_des=[]
#Slice_LUTs2_des=LUT_as_Logic2_des
Slice_LUTs2_des=[]
LUTs_difference_des=[]



for des in designs:
    
    
    var=0
    for fl in file:
        with open("Impl/{}/{}".format(des,fl)) as f:
            lines = f.read()
            sp = lines.split("\n")
            
            for l in sp:
                if fl == "power.txt":
                    
                    if "Signals" in l:
                        sig=float(l.split()[3])
                        sigdes.append(sig)
                        
                    #logic power
                    if "Slice Logic" in l:
                        slog=float(l.split()[4])
                        slogdes.append(slog)
                    
                    
                    if "Static Power" in l:
                        stp =float(l.split()[4])
                        stpdes.append(stp)
                        #print(l)
                        #rint(stp)
                    
                    if "Total On-Chip Power" in l:
                        tonp = float(l.split()[6])
                        tonpdes.append(tonp)
                        dc_power = tonp-stp
                        # print(tonp)
                        #print(stp)
                        # print(dc_power)
                        dc_power_des.append(dc_power)

#                     if ("Total On-Chip Power" or "Static Power") in l:
#                         tonp = float(l.split()[6])
#                         tonpdes.append(tonp)
#                         stp =float(l.split()[4])
#                         stpdes.append(stp)
#                         dc_power = tonp - stp
#                         dc_power_des.append(dc_power)
                            
                        
                if fl == "timing.txt":
                    if "Data Path Delay" in l:
                        #dpd = float(l.split()[3][:-2]) removed 2 to include ns unit
                        dpd = l.split()[3]
                        
                        dpddes.append(dpd)
                        
                if fl == "utilization.txt":
                    
                
                    if "F7 Muxes" in l:
                        mux= int(l.split()[4])
                        muxdes.append(mux)
                        
                    if "|   LUT as Memory" in l:
                        lutm = int(l.split()[5])
                        lutmdes.append(lutm)
                        
                    if "|   LUT as Logic" in l:
                        lutl = int(l.split()[5])
                        lutldes.append(lutl)
                    
                    if "Slice LUTs" in l:
                        sl = int(l.split()[4])
                        sldes.append(sl)
                        
                    if "LUT1"  in l:
                        lut1 = int(l.split()[3])
                        lut1des.append(lut1)
                        var=var+lut1
                        #.append(lut1) 
                        
                    if "LUT2" in l:
                        lut2 = int(l.split()[3])
                        lut2des.append(lut2)
                        var=var+lut2
                        #t.append(lut2)
                        
                    if "LUT3" in l:
                        lut3 = int(l.split()[3])
                        lut3des.append(lut3)
                        var=var+lut3
                        #t.append(lut3)
                        
                    if "LUT4" in l:
                        lut4 = int(l.split()[3])
                        lut4des.append(lut4)
                        var=var+lut4
                        #t.append(lut4)
                        
                    if "LUT5" in l:
                        lut5 = int(l.split()[3])
                        lut5des.append(lut5)
                        var=var+lut5
                        
                        #t.append(lut5)
                        
                    if "LUT6" in l:
                        lut6 = int(l.split()[3])
                        lut6des.append(lut6)
                        var=var+lut6
                        #t.append(lut6)
                        
                    if "LUT7" in l:
                        lut7 = int(l.split()[3])
                        lut7des.append(lut7)
                        var=var+lut7
                        #t.append(lut7)
                           
                        
                    if "| LUT as Memory" in l:
                         lutm2 = int(l.split()[5])
                         lutmdes2.append(lutm2)
    LUT_as_Logic2_des.append(var)
    
    Slice_LUTs2_des.append(var) 
    
    LUTs_difference =  var-sl
    LUTs_difference_des.append(LUTs_difference)
                    

In [574]:

result=[sigdes,slogdes,stpdes,dc_power_des,tonpdes,dpddes,muxdes,lutmdes,lutldes,sldes,lut1des,lut2des,lut3des,lut4des,lut5des,lut6des,lut7des,lutmdes2,LUT_as_Logic2_des,Slice_LUTs2_des,LUTs_difference_des]

df=pd.DataFrame(result)
#to replace Nan values with 0
df=df.fillna(0)
df_transpose=df.T
df_transpose.columns = col
df_transpose.insert(loc =0,column = 'File Name',value = designs)
df_transpose.to_csv("Result_final1.csv",index = None)
df_transpose

File Name Signal Power (w) Logic Power (w) Static Power (w) DC Power (w)  \
0   top_CLA8             0.17           0.066            0.191        2.597   
1  top_CLA16            0.496           0.192            0.256        10.95   
2  top_CLA32            1.724           0.507            0.514       22.963   
3  top_CLA64            4.872           1.148            2.405       48.738   

  Total Power (w) DataPath Delay F7_Muxes LUT_as_Memory LUT_as_logic  ...  \
0           5.002        6.927ns        0             0           13  ...   
1          11.141        8.335ns        0             0           35  ...   
2          23.219       11.285ns        0             0           97  ...   
3          49.252       16.020ns        0             0          233  ...   

  LUT2 LUT3 LUT4 LUT5 LUT6 LUT7 LUT_as_Memory LUT_as_Logic2 Slice_LUTs2  \
0    1    2    3    3    6    0             0            15          15   
1    6    4    8    5   22    0             0            45          45   
2   26    9   14   15   59    0             0           123         123   
3   48   14   25   44  155    0             0           286         286   

  LUTs_difference  
0               2  
1              10  
2              26  
3              53  

[4 rows x 22 columns]